In [1]:
import os
import pickle
import numpy as np
import pandas as pd
cwd = os.getcwd()

In [2]:
csv_file_path = '/home/nogaschw/Codeworkout/cleaned_code.csv'
df = pd.read_csv(csv_file_path, sep=',')
df = df[df.columns[1:]]

In [3]:
cleaned_file_path = '/home/nogaschw/Codeworkout/cleaned_questions.csv'

questions = pd.read_csv(cleaned_file_path, sep=',')
questions = questions[questions.columns[1:]]
questions = questions[questions['max_score'] != 0]
questions['prompt'] = questions['prompt'].apply(lambda x: x.split('PROBLEM STATEMENT:')[-1] if x.__contains__("PROBLEM STATEMENT:") else x) # clean question

In [4]:
df['problem_id'] = df['problem_id'].apply(lambda x: x.lower())
questions['id'] = questions['id'].apply(lambda x: x.lower())

In [5]:
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S')
df = df.sort_values('timestamp', ascending=True)
df = df.groupby(['student_id', 'problem_id', 'course_id']).apply(lambda x: pd.Series({
    'ServerTimestamp': x['timestamp'].tolist(),
    'score': x['score'].tolist(),
    'source_code': x['clean_code'].tolist()
})).reset_index()

/tmp/ipykernel_3661160/3206372099.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(['student_id', 'problem_id', 'course_id']).apply(lambda x: pd.Series({


In [6]:
def trim_after_max(score, snapshots, ServerTimestamp):
    max_index = len(score) - 1 - score[::-1].index(max(score))
    return score[:max_index+1], snapshots[:max_index+1], ServerTimestamp[:max_index+1]

# Apply the function to each row
df['score'], df['source_code'], df['ServerTimestamp'] = zip(*df.apply(lambda row: trim_after_max(row['score'], row['source_code'], row['ServerTimestamp']), axis=1))
df['start_time'] = [i[0] for i in df['ServerTimestamp']]
df['end_time'] = [i[-1] for i in df['ServerTimestamp']]
df['max_score'] = [max(i) for i in df['score']]
df['num_snapshots'] = df['source_code'].apply(lambda x: len(x))

In [7]:
percentiles = df.groupby('problem_id')['num_snapshots'].apply(lambda x: np.percentile(x, 75)).reset_index()
percentiles.columns = ['problem_id', '75th_percentile']
df = df.merge(percentiles, on='problem_id')
df['below_75th_percentile'] = df.apply(lambda row: row['num_snapshots'] <= row['75th_percentile'], axis=1)
df['correct_eventually'] = df.apply(lambda row: row['max_score'] >= 100, axis=1)

In [8]:
df['Label'] = df.apply(lambda row: row['correct_eventually'] & row['below_75th_percentile'], axis=1)

In [9]:
df['Label'].value_counts()

Label
True     90387
False    64677
Name: count, dtype: int64

In [10]:
print(len(df[df['max_score'] == -1]))
df = df[df['max_score'] != -1] # remove all the rows contain only locall code (not really the grade)

12353


In [11]:
df_basic = df.merge(questions, how='inner', left_on=['problem_id', 'course_id'], right_on=['id', 'course_id'])

In [12]:
df_basic = df_basic[df_basic['type'] != 'project']

In [13]:
df_basic['Label'].value_counts()

Label
True     81818
False    46860
Name: count, dtype: int64

In [14]:
df_basic = df_basic[['student_id', 'problem_id', 'course_id', 'type', 'start_time', 'end_time', 'score', 'source_code', 'prompt', 'correct_eventually','below_75th_percentile', 'num_snapshots', 'Label']]

In [15]:
df_sorted = df_basic.sort_values(by=['student_id', 'end_time'])
df_sorted['row_number'] = df_sorted.groupby(['student_id', 'course_id']).cumcount() + 1

In [16]:
prev_code = df_sorted['source_code'].tolist()
prev_code.insert(0, [])
prev_q = df_sorted['prompt'].tolist()
prev_q.insert(0, [])
prev_num_q = df_sorted['problem_id'].tolist()
prev_num_q.insert(0, [])
prev_score = df_sorted['score'].tolist()
prev_score.insert(0, [])
prev_label = df_sorted['Label'].tolist()
prev_label.insert(False, [])

In [17]:
df_sorted['prev_code'] = prev_code[:-1] # last question of the last student
df_sorted['prev_question'] = prev_q[:-1]
df_sorted['prev_question_num'] = prev_num_q[:-1]
df_sorted['prev_score'] = prev_score[:-1]
df_sorted['prev_label'] = prev_label[:-1]

df_sorted = df_sorted[df_sorted['row_number'] != 1]

In [18]:
prev_code = []
prev_question = []
prev_q_num = []
prev_score = []
prev_label = []
for i, row in df_sorted.iterrows():
    if row['row_number'] == 2:
        prev_code.append([row['prev_code']])
        prev_question.append([row['prev_question']])
        prev_q_num.append([row['prev_question_num']])
        prev_score.append([row['prev_score']])
        prev_label.append([row['prev_label']])
    else:
        prev_code.append(prev_code[-1] + [row['prev_code']])
        prev_question.append(prev_question[-1] + [row['prev_question']])
        prev_q_num.append(prev_q_num[-1] + [row['prev_question_num']])
        prev_score.append(prev_score[-1] + [row['prev_score']])
        prev_label.append(prev_label[-1] + [row['prev_label']])

In [19]:
df_sorted['prev_code'] = prev_code
df_sorted['prev_question'] = prev_question
df_sorted['prev_question_num'] = prev_q_num
df_sorted['score'] = prev_score
df_sorted['prev_label'] = prev_label

# Taxonomy

In [20]:
q_with_compu = questions.copy()
q_with_compu['IfElse'] = questions['conditional']
q_with_compu['Loops'] = questions[["loop_counting", "loop_until", "loop_elements", "loop_nested"]].max(axis=1)
q_with_compu['MathOperations'] = questions[["stat_calculate", "assignment"]].max(axis=1)
q_with_compu['LogicOperators'] = 0
q_with_compu['StringOperations'] = questions[["input_str", "input_cast", "output"]].max(axis=1)
q_with_compu["List"]= questions[["list", "list_2d"]].max(axis=1)
q_with_compu['FileOperations'] = questions[["file_read", "file_write"]].max(axis=1)
q_with_compu["Functions"] = questions[["function_call", "function_def", "function_return"]].max(axis=1)
q_with_compu['Dictionary'] = questions["dictionary"]
q_with_compu['Tuple'] = questions[["tuple", "item_set"]].max(axis=1)

q_with_compu = q_with_compu.drop(columns=['input_str', 'input_cast', 'output', 'assignment', 'conditional',
       'function_call', 'function_def', 'function_return', 'loop_counting',
       'loop_until', 'loop_elements', 'loop_nested', 'stat_calculate',
       'file_read', 'file_write', 'list', 'list_2d', 'dictionary', 'item_set',
       'tuple'])


q_with_compu.fillna(0, inplace=True)
questions.fillna(0, inplace=True)

In [21]:
q_with_compu = q_with_compu.rename(columns={'prompt': 'question'})

In [34]:
df_taxonomy = df_sorted.merge(q_with_compu.drop(columns=['type']), how='left', left_on=['problem_id', 'course_id'], right_on=['id', 'course_id'])

In [36]:
df_taxonomy = df_taxonomy[['student_id', 'course_id', 'problem_id', 'prev_code', 'prev_question', 'prev_question_num', 'score', 'question', 'prev_label', 'correct_eventually' ,'below_75th_percentile', 'num_snapshots', 'Label', 'type', 'IfElse', 'Loops',
       'MathOperations', 'LogicOperators', 'StringOperations', 'List', 'FileOperations', 'Functions', 'Dictionary', 'Tuple']]

In [22]:
# with open(f'/home/nogaschw/Codeworkout/Thesis/Data/falcon/split_ids_valid_ids.pkl', 'rb') as f:
#         valid_ids = pickle.load(f)
# with open(f'/home/nogaschw/Codeworkout/Thesis/Data/falcon/split_ids_test_ids.pkl', 'rb') as f:
#     test_ids = pickle.load(f)

In [24]:
# valid_df = df_taxonomy[df_taxonomy['student_id'].isin(valid_ids)]
# test_df = df_taxonomy[df_taxonomy['student_id'].isin(test_ids)]
# pd.concat([valid_df, test_df], ignore_index=True).to_pickle(os.path.join(cwd, '../df_taxonomy_falcon_test_valid.pkl'))

In [25]:
def check_snapshots_num():
    arr_snapshot_num = []
    max_snapshots = 0
    max_total = 0
    max_row_snapshots = None
    max_row_total = None
    for j, row in df_taxonomy.iterrows():
        r = row['prev_code']
        sum_snapshots = 0
        for i in r:
            sum_snapshots += len(i)
            arr_snapshot_num.append(len(i))
            if len(i) > max_snapshots:
                max_snapshots = len(i)
                max_row_snapshots = row
        if sum_snapshots > max_total:
            max_total = sum_snapshots
            max_row_total = row

In [37]:
df_taxonomy.to_pickle(os.path.join(cwd, '../df_taxonomy_falcon_new_label.pkl'))